In [1]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.6/390.6 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 12.2 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords, wordnet
from wordcloud import WordCloud, STOPWORDS
from nltk.stem import WordNetLemmatizer
import os
import string
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn.decomposition import LatentDirichletAllocation
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
# import plotly.express as px
from sklearn.manifold import TSNE
import itertools
import collections
from collections import Counter
from gensim import corpora, models
from gensim.models import Word2Vec, word2vec, LdaModel
import optuna
from hyperopt import hp, tpe, fmin
import warnings
warnings.filterwarnings("ignore")
import gensim
from gensim.utils import simple_preprocess
from datetime import datetime
# import spacy
from nltk import pos_tag
import pickle


In [3]:
nltk.download('averaged_perceptron_tagger')
lemmatizer = WordNetLemmatizer()
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
# nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('drive/MyDrive/5.TopicModeling/Data/final.csv', encoding='latin-1')

# Word Distribution

In [ ]:
wordcounter = Counter()
# Update the Counter with all tokens in the 'lemm_text' column
for tokens in df['lemm_text']:
    wordcounter.update(tokens)
df['lemm_textA'] = [[word for word in tokens if wordcounter[word] >= 100] for tokens in df['lemm_text']]
df

# Find dimension

In [13]:
# Create an instance of CountVectorizer
vectorizer = CountVectorizer(
                             max_features = 1000)

# document_texts = [' '.join(tokens) for tokens in df['lemm_text']]
document_texts = [tokens for tokens in df['lemm_text']]


# Fit the vectorizer on the 'lemm_text' column
vectorizer.fit(document_texts)

# Transform the 'lemm_text' column into a count vector representation
count_vector = vectorizer.transform(document_texts)

# Get the dimensions of the count vector
vector_dimensions = count_vector.shape

# Print the dimensions
print("Count Vector Dimensions:", vector_dimensions)


Count Vector Dimensions: (301476, 1000)


# Gensim LDA

In [5]:
# Create a list of lists from the preprocessed documents
# processed_docs = [doc.split() for doc in df['lemm_text']]
processed_docs = [re.sub(r'[\[\],]', '', doc).split() for doc in df['lemm_text']]
dictionary = corpora.Dictionary(processed_docs)
# dictionary.filter_extremes(no_below=5, no_above=0.5, keep_n=1000)  # Limit the dictionary size
# Convert the dictionary into a bag-of-words representation
corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
# Initialize the LDA model with 10 top
num_topics = 10
lda_model = models.LdaModel(corpus, num_topics=num_topics, id2word=dictionary, random_state = 5566)

In [17]:
# Increase the number of topics from 10 to 100
new_num_topics = 100

while num_topics < new_num_topics:
    # Increase the number of topics
    num_topics += 10

    # Re-train the LDA model with the updated number of topics
    lda_model = models.LdaModel(corpus, num_topics=num_topics, id2word=dictionary)

    # Evaluate the model's performance (using coherence score as an example)
    coherence_score = models.CoherenceModel(model=lda_model, texts=processed_docs, dictionary=dictionary, coherence='c_v').get_coherence()

    print(f"Number of topics: {num_topics}, Coherence Score: {coherence_score}")

    # Repeat the process until the desired number of topics or satisfactory results are achieved

Number of topics: 20, Coherence Score: 0.2967513086278536
Number of topics: 30, Coherence Score: 0.2646094798249947
Number of topics: 40, Coherence Score: 0.28905500144687324
Number of topics: 50, Coherence Score: 0.3047479493019113
Number of topics: 60, Coherence Score: 0.3341418134922695
Number of topics: 70, Coherence Score: 0.34743565677642696
Number of topics: 80, Coherence Score: 0.3163543212833361
Number of topics: 90, Coherence Score: 0.28777561040677907
Number of topics: 100, Coherence Score: 0.2703842819484554


In [ ]:
#try new topic model
lda_modelA = models.LdaModel(corpus, num_topics=70, id2word=dictionary)
for i,topic in lda_modelA.show_topics(formatted=True, num_topics=70, num_words=15):
    print(str(i)+": "+ topic)
    print()

#Parameter Modification (Optuna)

In [ ]:
def objective(trial):
    num_topics = trial.suggest_int('num_topics', 10, 100)
    # iterations = trial.suggest_int('iterations', 50, 100)
    alpha = trial.suggest_categorical('alpha', ['symmetric', 'asymmetric'])
    eta = trial.suggest_float('eta', 0.1, 1.0)
    # rounded_eta = round(suggested_eta, 2)

    # Handle None values for hyperparameters
    # alpha = 1.0 if alpha is None else alpha
    # beta = 1.0 if beta is None else beta

    # Train the LDA model with the current hyperparameters
    lda_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics,
                         alpha=alpha, eta=eta)

    # Calculate coherence score for the model
    coherence_model = models.CoherenceModel(model=lda_model, texts=processed_docs, dictionary=dictionary, coherence='c_v')
    coherence_score = coherence_model.get_coherence()

    return coherence_score

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

# Print the best hyperparameters and coherence score
best_params = study.best_params
best_coherence = study.best_value
print("Best number of topics:", best_params['num_topics'])
print("Best alpha:", best_params['alpha'])
print("Best beta:", best_params['eta'])
print("Best coherence score:", best_coherence)


In [26]:
best_params = study.best_params
best_coherence = study.best_value
print("Best number of topics:", best_params['num_topics'])
print("Best alpha:", best_params['alpha'])
print("Best beta:", best_params['eta'])
print("Best coherence score:", best_coherence)

Best number of topics: 63
Best alpha: asymmetric
Best beta: 0.9886469911764294
Best coherence score: 0.41332203244319493


In [ ]:
modified_lda_model = LdaModel(corpus, id2word = dictionary,
                       num_topics = best_params['num_topics'], alpha = best_params['alpha'],
                       eta = best_params['eta'])
for i,topic in modified_lda_model.show_topics(formatted=True, num_topics=best_params['num_topics'], num_words=15):
    print(str(i)+": "+ topic)
    print()

In [28]:
 # save model
# with open('drive/MyDrive/5.TopicModeling/Model/lda_wholeperiod_mod', 'wb') as file:
    # pickle.dump(modified_lda_model, file)

In [6]:
# Load the LDA model
model_path = 'drive/MyDrive/5.TopicModeling/Model/lda_wholeperiod_mod'
with open(model_path, 'rb') as file:
    modified_lda_model = pickle.load(file)

#implement model
for i,topic in modified_lda_model.show_topics(formatted=True, num_topics=63, num_words=15):
    print(str(i)+": "+ topic)
    print()

0: 0.251*"'smoke'" + 0.046*"'day'" + 0.014*"'tire'" + 0.013*"'cigarette'" + 0.013*"'second'" + 0.012*"'hand'" + 0.011*"'depression'" + 0.009*"'smell'" + 0.008*"'make'" + 0.008*"'wild'" + 0.007*"'blunt'" + 0.007*"'blow'" + 0.006*"'high'" + 0.006*"'mirror'" + 0.005*"'ash'"

1: 0.105*"'fire'" + 0.019*"'get'" + 0.017*"'want'" + 0.013*"'terrible'" + 0.012*"'amp'" + 0.010*"'worry'" + 0.010*"'people'" + 0.009*"'make'" + 0.008*"'time'" + 0.008*"'say'" + 0.008*"'tire'" + 0.008*"'angry'" + 0.007*"'year'" + 0.007*"'go'" + 0.007*"'see'"

2: 0.125*"'california'" + 0.109*"'wildfire'" + 0.090*"'large'" + 0.079*"'history'" + 0.070*"'state'" + 0.041*"'become'" + 0.012*"'los'" + 0.012*"'angeles'" + 0.006*"'modern'" + 0.006*"'hop'" + 0.005*"'emergency'" + 0.005*"'rag'" + 0.005*"'third'" + 0.004*"'thousand'" + 0.004*"'space'"

3: 0.104*"'california'" + 0.063*"'fire'" + 0.039*"'wildfire'" + 0.038*"'firefighter'" + 0.028*"'southern'" + 0.024*"'wind'" + 0.018*"'blaze'" + 0.016*"'news'" + 0.016*"'struggle'" +

# Topic Definition

### Topic 0: Concerns about Smoke

The most prominent keyword is "smoke" with a weight of 0.251, indicating that it is the central theme of the topic. Other related keywords like "day," "cigarette," "second-hand," and "smell" further support the focus on smoke-related matters.

### Topic 9: Emotional State and Tragic Events

The prominent keyword "'sad'" indicates a focus on feelings of sadness, while the term "'sick'" suggests a state of physical or emotional distress. Other keywords like "'deadly'" and "'mumbai'" hint at the presence of tragic events or situations.

The mention of "'hero'" and "'camp'" may indicate some context related to heroic acts or challenging circumstances. Additionally, terms like "'okay'," "'sometimes'," and "'quite'" suggest fluctuations in emotions or experiences.

### Topic 19: Air Quality and Health Concerns

The keywords such as "'air'," "'tragedy'," "'health'," and "'quality'" suggest a focus on issues related to air pollution and its impact on health. Additionally, words like "'victim'," "'app'," "'wear'," and "'mask'" further support the idea of discussing measures to protect oneself from air pollution and its potential health consequences. The presence of location-specific terms like "'mumbai'" might indicate a particular context or situation related to air quality in Mumbai.

### Topic 20: Reasons for Visiting Napa and Sonoma Valley

The keywords such as "'reason'", "'apartment'", "'napa'", "'sonoma'", "'valley'", "'winery'", "'dinner'", and "'beer'" suggest a discussion about visiting or staying in Napa and Sonoma Valley. Other words like "'incredibly'", "'robert'", "'strike'", and "'smokey'" may indicate additional details or experiences related to the topic. However, the prominence of the location-specific terms strongly suggests that the topic revolves around reasons for visiting Napa and Sonoma Valley.

### Topic 23: Tax-related Stress and Relief

The keywords such as "'tax'," "'bill'," and "'stress'" indicate that the topic revolves around tax-related concerns and the associated stress. The word "'glad'" suggests a potential relief or positive aspect related to taxes. The presence of words like "'senator'" and "'announce'" might indicate some political context or a public statement related to taxes. Overall, the topic seems to focus on the stress and relief associated with taxes, potentially involving discussions about tax bills, announcements, and political figures.

### Topic 30: Federal Regulations in the Medical Field

Based on the provided keywords, it appears that the topic is related to federal and medical matters. The keywords "'federal'" and "'medical'" are the most significant in the list, suggesting a focus on these subjects. Other keywords like "'steve,'" "'rough,'" "'keith,'" and various terms related to nutrition and substances ('vit,' 'nutrition,' 'cannibis,' 'glutathione,' 'bioflavanoids,' 'tumeric') may not contribute significantly to the main topic.

### Topic 31: Anxiety and its Effects on Mental Health

The prominent keyword "anxiety" indicates that the topic revolves around anxiety itself. Other keywords like "face," "word," "imagine," and "toxic" suggest the emotional and psychological impact of anxiety on individuals. The inclusion of terms like "cnn" and "retweet" may imply discussions about anxiety in the context of social media or public discourse. Overall, the topic seems to focus on anxiety and its effects on mental health, including potential related disorders such as PTSD.

### Topic 35: Twitter's Handling of Abusive Content.

The presence of words like "allow," "information," and "target" suggests a focus on Twitter's policies and practices. The mention of "abusive" indicates that the topic might involve discussions related to addressing abusive behavior or content on the platform. While the remaining keywords don't provide much context, they could be related to specific incidents, locations, or usernames mentioned in the discussions regarding Twitter's policies.

### Topic 37: Action and Decision-Making

The keywords like "'action'," "'join'," and "'experience'" suggest an involvement in activities or events. The word "'decide'" implies the process of making choices or decisions. Additionally, the word "'mental'" might indicate a mental aspect related to the topic, such as psychological considerations or mindset.



### Topic 39: Political Discussions and Criminal Justice

The presence of words like "'political'," "'friends'," "'calif'," and "'criminal'" indicates a discussion related to politics and the criminal justice system. The keywords "'review'," "'movement'," and "'union'" also suggest a potential focus on evaluating or critiquing certain aspects. The inclusion of words like "'whistle'" and "'diversity'" may indicate discussions related to activism or social issues within the political context.

### Topic 60: Santa Rosa Fireworks

The keywords "'santa'," "'rosa'," and "'firework'" suggest a connection to a location named Santa Rosa and the specific event of fireworks. Other keywords such as "'town'," "'cook'," and "'empty'" might indicate additional details or context related to the topic.


